In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, GridSearchCV
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error

In [2]:
# # import cdr data
# cdr_data = pd.read_csv("data/cdr_all_feats_data_new_split.csv")
# gcn_data = pd.read_csv("data/gcn_all_feats_data_new_split.csv")

In [3]:
test_index = 1

In [4]:
deepcdr_preds = np.load("..//data//ablation_deepcdr_preds_val_and_test_" +str(test_index) + ".npy")

In [5]:
dualgcn_preds = np.load("..//data//ablation_dualgcn_complete_preds_val_and_test_" +str(test_index) + ".npy")

In [6]:
np.mean(deepcdr_preds[:,0] == dualgcn_preds[:,0])

1.0

In [7]:
deepcdr_preds.shape

(11544, 26)

In [8]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [9]:
chunks = np.array_split(list(range(0,deepcdr_preds.shape[0])), 2)

In [10]:
half = 1

In [11]:
train_items = chunks[half]

In [12]:
test_items = list(set(list(range(0,deepcdr_preds.shape[0]))).difference(train_items))

In [13]:
# dualgcn_preds[:,0]

In [14]:
# cdr_data.columns = ['True_response', 'Predicted_y_cdr']

In [15]:
valid_deepcdr = deepcdr_preds[train_items,1:]
test_deepcdr = deepcdr_preds[test_items,1:]

In [16]:
valid_dualgcn = dualgcn_preds[train_items,1:]
test_dualgcn = dualgcn_preds[test_items,1:]

In [17]:
# valid_dualgcn.shape

In [18]:
valid_y = deepcdr_preds[train_items,0]
test_y = deepcdr_preds[test_items,0]

In [19]:
# stacker_data_train.shape

In [20]:
test_y

array([3.143044, 3.213836, 1.036346, ..., 3.244352, 3.47219 , 3.39551 ])

In [21]:
test_y.mean()

2.069332453915454

In [22]:
test_y.std()

2.8460217894549977

In [23]:
reps = valid_deepcdr.shape[1]

In [24]:
catch_preds = np.zeros((test_dualgcn.shape[0],reps))
catch_weights = np.zeros((reps,2))
for i in range(0,valid_deepcdr.shape[1]):
    stacker_data_train = np.hstack((valid_deepcdr[:,i].reshape(-1,1), valid_dualgcn[:,i].reshape(-1,1)))
    stacker_data_test = np.hstack((test_deepcdr[:,i].reshape(-1,1), test_dualgcn[:,i].reshape(-1,1)))
    reg = LinearRegression()
    reg.fit(stacker_data_train, valid_y)
    y_preds = reg.predict(stacker_data_test)
    res = pearsonr(test_y.tolist(), y_preds.tolist())
    catch_preds[:,i] = y_preds
    catch_weights[i,0], catch_weights[i,1] = reg.coef_[0]/(reg.coef_[0]+reg.coef_[1]), reg.coef_[1]/(reg.coef_[0]+reg.coef_[1])
    # print(res[0])
    # print(np.sqrt(mean_squared_error(test_y, y_preds)))
    # plt.scatter(y_preds.reshape(-1,1), test_y, alpha = 0.1)
    # plt.axline((0,0), slope = 1, c = "black")
    # plt.show()

In [25]:
# test_y

In [26]:
catch_preds.shape

(5772, 25)

In [27]:
li, ui = np.percentile(catch_preds, q = [2.5, 97.5], axis = 1)[0,:], np.percentile(catch_preds, q = [2.5, 97.5], axis = 1)[1,:]

In [28]:
test_coverage = np.mean((test_y.reshape(-1,1) >= li.reshape(-1,1)) & (test_y.reshape(-1,1) <= ui.reshape(-1,1)))
test_coverage

0.4203049203049203

In [29]:
test_width = np.mean(ui.reshape(-1,1) - li.reshape(-1,1))
test_width

1.3399442968710935

In [30]:
catch_preds.mean(1).shape

(5772,)

In [31]:
test_y.shape

(5772,)

In [32]:
test_pearsonr = pearsonr(test_y, catch_preds.mean(1))
test_pearsonr[0]

0.8844630311215945

In [33]:
test_rmse = np.sqrt(np.mean((catch_preds.mean(1).reshape(-1,1)-test_y.reshape(-1,1))**2))
test_rmse

1.3297560768519565

In [34]:
catch_weights

array([[0.6308114 , 0.3691886 ],
       [0.59967835, 0.40032165],
       [0.61160605, 0.38839395],
       [0.61944765, 0.38055235],
       [0.59636042, 0.40363958],
       [0.61483738, 0.38516262],
       [0.59054505, 0.40945495],
       [0.61499938, 0.38500062],
       [0.59384937, 0.40615063],
       [0.61327069, 0.38672931],
       [0.61085426, 0.38914574],
       [0.61816443, 0.38183557],
       [0.60630311, 0.39369689],
       [0.62626865, 0.37373135],
       [0.59027904, 0.40972096],
       [0.59589968, 0.40410032],
       [0.60345589, 0.39654411],
       [0.59771702, 0.40228298],
       [0.61000347, 0.38999653],
       [0.60408098, 0.39591902],
       [0.60448398, 0.39551602],
       [0.61738841, 0.38261159],
       [0.60608579, 0.39391421],
       [0.62397912, 0.37602088],
       [0.61228629, 0.38771371]])

In [35]:
test_cdr_weight, test_gcn_weight = np.mean(catch_weights,0)[0], np.mean(catch_weights,0)[1]
test_cdr_weight, test_gcn_weight

(0.6085062334756488, 0.39149376652435114)

In [36]:
np.std(catch_weights,0)

array([0.01081359, 0.01081359])

In [37]:
# save the metrics for later\
col_name = ['test_coverage', 'test_width', 'test_pearsonr', 'test_rmse', 'test_cdr_weight', 'test_gcn_weight' ]
col_vals = [test_coverage, test_width, test_pearsonr[0], test_rmse, test_cdr_weight, test_gcn_weight]

In [38]:
fold_idx = 0

In [39]:
Stacker_test_results = pd.DataFrame({'parameter': col_name, 'value': col_vals})
save_name = 'test_parameters_ablation_stacker_complete_for_test_fold_' + str(fold_idx) + '.csv'
save_path = '../MEnKF_fold_results/Fold_0/'
Stacker_test_results.to_csv(save_path + save_name, index = False)

In [40]:
Stacker_test_results

,parameter,value
0,test_coverage,0.420305
1,test_width,1.339944
2,test_pearsonr,0.884463
3,test_rmse,1.329756
4,test_cdr_weight,0.608506
5,test_gcn_weight,0.391494
